# **Proyecto Final**
## Asignatura: Visualización de Datos
## Curso: 2023/2024
### Alumna: Lydia Ruiz Martínez
### Correo: 202213363@alu.comillas.edu


Primero importamos las librerías y módulos necesarios para construir nuestra aplicación web interactiva utiliando Dash.

En esta app vamos a estudiar cómo ha sido el año a nivel de ventas mediante la obtención de manera visual de los principales KPIs de la empresa de alimentación de la cual vamos a estudiar los datos.

In [2]:
# Importamos las librerías
import dash
from dash import dash_table
# Importamos dash_html_components como html
from dash import html
from dash import dcc

# Importamos dash_core_components como dcc
from dash.dependencies import Input,Output
import pandas as pd
import plotly.express as px

# Importamos jupyter dash para trabajar en el notebook
from dash import jupyter_dash
from dash import Dash

Utilizamos Google Colab para montar Google Drive en el entorno de ejecución.

In [3]:
# El notebook se ejecuta desde Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Leemos los datos, los concatenamos (ya que hay dos datasets distintos) y visualizamos las primeras filas para ver qué aspectos tienen los datos con los que vamos a trabajar.

In [4]:
# Cargamos los datasets y los concatenamos para trabajar con el dataframe (df)

parte_1 = pd.read_csv('/content/drive/MyDrive/PROYECTO FINAL VISU/data/parte_1.csv',sep = ',',index_col=[0]).reset_index()
parte_2 = pd.read_csv('/content/drive/MyDrive/PROYECTO FINAL VISU/data/parte_2.csv',sep = ',',index_col=[0]).reset_index()

df = pd.concat([parte_1, parte_2])

df.head()

<ipython-input-4-cb4fb2d9d831>:3: DtypeWarning: Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  parte_1 = pd.read_csv('/content/drive/MyDrive/PROYECTO FINAL VISU/data/parte_1.csv',sep = ',',index_col=[0]).reset_index()
<ipython-input-4-cb4fb2d9d831>:4: DtypeWarning: Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  parte_2 = pd.read_csv('/content/drive/MyDrive/PROYECTO FINAL VISU/data/parte_2.csv',sep = ',',index_col=[0]).reset_index()


,index,id,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,...,city,state,store_type,cluster,transactions,year,month,week,quarter,day_of_week
0,0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Ecuador,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
1,1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,National,Ecuador,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
2,2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,National,Ecuador,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
3,3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,National,Ecuador,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
4,4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,National,Ecuador,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday


Inicializamos la aplicación Dash y configuramos el título de nuestra aplicación.

In [5]:
# Iniciamos la aplicación Dash

app= Dash(__name__, suppress_callback_exceptions=True)

app.title = "Trabajo Final"

Procedemos a definir estilos para las pestañas de la aplicación Dash.

In [6]:
# Definimos estilos para las pestañas

tab_style = {
    'border': '1px solid #A1E5A4',
    'padding': '6px',
    'backgroundColor': '#F0F8FF',
    'color': '#000000',
}

tab_selected_style = {
    'border': '1px solid #A1E5A4',
    'padding': '6px',
    'backgroundColor': '#2D572C',
    'color': '#FFFFFF',
}

Definimos varias funciones para generar distintas gráficas en función de los datos que vamos a estudiar en cada pestaña.

Para dibujarlas vamos a utilizar Plotly Express.

Estas gráficas se van a visualizar en las pestañas 1 y 4

In [7]:
def grafica_1():
    # Gráfica con el conteo general de los datos (PESTAÑA 1.a)
    num_tiendas = df['store_nbr'].nunique()
    num_productos = df['family'].nunique()
    num_estados = df['locale_name'].nunique()
    num_meses = df['month'].nunique()
    figure = px.bar(x=['Tiendas', 'Productos', 'Estados', 'Meses'],
                      y=[num_tiendas, num_productos, num_estados, num_meses], color = ['Tiendas', 'Productos', 'Estados', 'Meses'],
                      labels={'x': 'Datos', 'y': 'Número total'},
                      title='Conteo general de tiendas, productos, estados y meses')
    return figure

def grafica_2():
    # Calculamos el ranking de productos por la media de ventas y seleccionamos los 10 primeros en orden descendente (PESTAÑA 1.b.i)
    ranking_productos = df.groupby('family')['sales'].mean().sort_values(ascending = False).reset_index()
    ranking_productos = ranking_productos.head(10)
    figure=px.bar(x=ranking_productos['family'], y=ranking_productos['sales'], color=ranking_productos['family'],
                      labels={'x': 'Tipo de producto', 'y': 'Número de ventas'},
                      title='Ranking (top 10) de los productos más vendidos', color_continuous_scale='set1')
    return figure

def grafica_3():
    # Calculamos la distribución de ventas por tiendas (PESTAÑA 1.b.ii)
    dist_ventas_tiendas = df.groupby('store_nbr')['sales'].mean().reset_index()
    figure=px.bar(x=dist_ventas_tiendas['store_nbr'], y=dist_ventas_tiendas['sales'],
                      labels={'x': 'Tiendas', 'y': 'Número de ventas'},
                      title='Distribución de las ventas por tiendas')
    return figure

def grafica_4():
    # Calculamos el ranking de productos en promoción por la media de ventas y seleccionamos los 10 primeros en orden descendente (PESTAÑA 1.b.iii)
    df_promocion = df[['store_nbr','sales','onpromotion']]
    df_promocion['onpromotion'] = df_promocion['onpromotion'].apply(lambda x: None if x!= True else x)
    df_promocion = df_promocion.dropna()
    df_promocion = df_promocion.groupby('store_nbr')['sales'].mean().sort_values(ascending = False).reset_index()
    df_promocion['store_nbr'] = df_promocion['store_nbr'].astype(str)
    df_promocion = df_promocion.head(10)
    figure=px.bar(x=df_promocion['store_nbr'], y=df_promocion['sales'],color=df_promocion['store_nbr'],
                      labels={'x': 'Tiendas', 'y': 'Número de ventas'},
                      title='Ranking (top 10) de tiendas con ventas en productos en promoción')
    return figure

def grafica_5():
    # Calculamos las ventas medias de cada día (PESTAÑA 1.c.i)
    semana_ventas = df.groupby('day_of_week')['sales'].mean().sort_values(ascending=False).reset_index()
    colors = ['blue' if day == 'Sunday' else 'pink' for day in semana_ventas['day_of_week']]
    figure = px.pie(
        names=semana_ventas['day_of_week'],
        values=semana_ventas['sales'],
        labels={'day_of_week': 'Día de la semana', 'sales': 'Número de ventas'},
        title='El día de la semana en el que se producen más ventas es el domingo'
    )
    figure.update_traces(marker=dict(colors=colors))

    return figure

def grafica_6():
    # Calculamos las ventas medias de cada semana de los años del dataset (PESTAÑA 1.c.ii)
    semana_ventas_year = df.groupby('week')['sales'].mean().reset_index()
    figure=px.line(x=semana_ventas_year['week'], y=semana_ventas_year['sales'],
                      labels={'x': 'Número de semana', 'y': 'Número de ventas'},
                      title='Volumen de ventas medio por semana del año')
    return figure

def grafica_7():
    # Calculamos las ventas medias de cada mes de los años del dataset (PESTAÑA 1.c.iii)
    mes_ventas_year = df.groupby('month')['sales'].mean().reset_index()
    figure=px.line(x=mes_ventas_year['month'], y=mes_ventas_year['sales'],
                      labels={'x': 'Número de mes', 'y': 'Número de ventas'},
                      title='Volumen de ventas medio por mes del año')
    return figure

def grafica_8():
    # Evolución temporal de las ventas de la empresa (PESTAÑA 4)
    figure = px.line(
        x=df.groupby('date')['sales'].sum().index,
        y=df.groupby('date')['sales'].sum(),
        labels={'x': 'Fecha', 'y': 'Número de ventas (medio)'},
        title='Evolución de las ventas de la empresa'
    )
    return figure

def grafica_9():
    # Calculamos las ventas totales por estado (PESTAÑA 4)
    ventas_por_estado = df.groupby('state')['sales'].sum().reset_index()
    figure = px.bar(
        ventas_por_estado,
        x='state',
        y='sales',
        title='Ventas totales por estado',
        labels={'state': 'Estado', 'sales': 'Ventas Totales'},
        color='state'
    )

    return figure

def grafica_10():
    # Estudiamos la distribución mensual de ventas en las 5 categorías con más ventas (PESTAÑA 4)
    top_categories = df.groupby('family')['sales'].sum().nlargest(5).index
    df_top_categories = df[df['family'].isin(top_categories)]
    monthly_sales_top_categories = df_top_categories.groupby(['month', 'family'])['sales'].sum().reset_index()
    figure = px.line(
        monthly_sales_top_categories,
        x='month',
        y='sales',
        color='family',
        labels={'month': 'Número de mes', 'sales': 'Número de ventas'},
        title='Distribución mensual de ventas de las 5 categorías principales'
    )
    return figure


Ahora vamos a crear dos callbacks en la aplicación Dash.

El primero está relacionado con las tiendas de las que disponemos los datos y se visualiza en la pestña 2.

El segundo está relacionado con los estados de los que disponemos los datos y se visualiza en la pestaña 3.

In [8]:
# (APARTADOS 2.a, 2.b y 2.c)
@app.callback(
    [Output('total-sales', 'figure'),
     Output('total-products', 'figure'),
     Output('total-products-promotion', 'figure')],
    [Input('store-dropdown', 'value')]
)
def update_graph_tienda(tienda_elegida):
    # Filtramos datos por la tienda seleccionada
    tienda_filtrada = df[df['store_nbr'] == tienda_elegida]

    # Actualizamos gráfico de ventas por año
    ventas_por_ano = tienda_filtrada.groupby('year')['sales'].sum().reset_index().sort_values(by='year')
    figure1 = px.bar(x=ventas_por_ano['year'], y=ventas_por_ano['sales'],
                     labels={'x': 'Años', 'y': 'Número de ventas (medio)'},
                     title=f'Número total de ventas por año de la tienda {tienda_elegida}')

    # Actualizamos gráfico de productos vendidos
    productos_vendidos = tienda_filtrada.groupby('family')['sales'].sum().reset_index()
    figure2 = px.bar(x=productos_vendidos['family'], y=productos_vendidos['sales'], color=productos_vendidos['family'],
                     labels={'x': 'Productos', 'y': 'Número de ventas'},
                     title=f'Número total de productos vendidos en la tienda {tienda_elegida}')

    # Actualizamos gráfico de productos en promoción
    tienda_filtrada['onpromotion'] = tienda_filtrada['onpromotion'].apply(lambda x: None if x != True else x)
    tienda_filtrada_promo = tienda_filtrada.dropna().groupby('family')['sales'].sum().reset_index()
    figure3 = px.bar(x=tienda_filtrada_promo['family'], y=tienda_filtrada_promo['sales'],
                     color=tienda_filtrada_promo['family'],
                     labels={'x': 'Productos (En promoción)', 'y': 'Número de ventas'},
                     title=f'Número total de productos en promoción vendidos en la tienda {tienda_elegida}')

    return figure1, figure2, figure3

# (APARTADOS 3.a, 3.b y 3.c)
@app.callback(
    [Output('total-transactions', 'figure'),
     Output('ranking-store-sales', 'figure'),
     Output('most-selled-product', 'figure')],
    [Input('state-dropdown', 'value')]
)
def update_graph_estado(estado_elegido):
    # Filtramos datos por el estado seleccionado
    estado_filtrado = df[df['state'] == estado_elegido]

    # Actualizamos el gráfico de transacciones por año
    estado_filtrado_1 = estado_filtrado[['year', 'sales', 'transactions']].dropna()
    estado_filtrado_1 = estado_filtrado_1.groupby('year')['transactions'].sum().reset_index().sort_values(by='year')
    figure1 = px.bar(x=estado_filtrado_1['year'], y=estado_filtrado_1['transactions'],
                     labels={'x': 'Años', 'y': 'Número de transacciones'},
                     title=f'Número total de transacciones por año en el estado de {estado_elegido}')

    # Actualizamos el gráfico de ranking de tiendas con más ventas
    estado_filtrado_2 = estado_filtrado.copy()
    estado_filtrado_2['store_nbr'] = estado_filtrado_2['store_nbr'].astype('str')
    estado_filtrado_2 = estado_filtrado_2.groupby('store_nbr')['sales'].sum().reset_index().sort_values(by='sales', ascending=False)
    figure2 = px.bar(x=estado_filtrado_2['store_nbr'], y=estado_filtrado_2['sales'], color=estado_filtrado_2['store_nbr'],
                      labels={'x': 'Tiendas', 'y': 'Número de ventas(Total)'},
                      title=f'Ranking de tiendas con más ventas en el estado de {estado_elegido}')

    # Actualizamos el gráfico de producto más vendido en cada tienda
    most_sold_product_per_store = estado_filtrado.groupby(['store_nbr', 'family'])['sales'].sum().reset_index()
    most_sold_product_per_store = most_sold_product_per_store.sort_values(by='sales', ascending=False)
    most_sold_product_per_store = most_sold_product_per_store.groupby('store_nbr').first().reset_index()
    figure3 = px.bar(x=most_sold_product_per_store['store_nbr'].astype(str),
                     y=most_sold_product_per_store['sales'],
                     color=most_sold_product_per_store['family'],
                     labels={'x': 'Tiendas', 'y': 'Número de ventas'},
                     title=f'Producto más vendido por tienda en el estado de {estado_elegido}')

    return figure1, figure2, figure3

Definimos el Layout general (contenedor html.Div).

Organizamos los gráficos y tablas en las pestañas donde queremos visualizarlas.

Definimos la función "render_content" para manejar el contenido de las 4 pestañas.

In [9]:
# Definimos el diseño de la aplicación

app.layout = html.Div([
    html.H2('Lydia Ruiz Martínez', style={
        'fontFamily': 'Times New Roman',
        'color': '#000000',
        'textAlign': 'right',
        'padding': '10px'
    }),
    html.H1(' Dashboard de KPIs - Empresa de Alimentación ', style={
        'textAlign': 'center',
        'color': '#000000',
        'background-color': '#A1E5A4',
        'padding': '20px',
        'border-radius': '10px'
    }),

    dcc.Tabs(id="tabs-example-graph", value='tab-1-example-graph', children=[
        dcc.Tab(label='Situación global de las ventas', value='tab_1', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Información por tienda', value='tab_2', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Información a nivel estado', value='tab_3', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Conclusiones de ventas', value='tab_4', style=tab_style, selected_style=tab_selected_style),
    ]),
    html.Div(id='tabs-content-example-graph')
],
    style={'backgroundColor': '#EFEFEF'})

@app.callback(Output('tabs-content-example-graph', 'children'),
              Input('tabs-example-graph', 'value'))

def render_content(tab):
    if tab == 'tab_1':
        return html.Div([
            html.H1('Visualización global de la situación de las ventas', style={'border': '2px solid #A1E5A4', 'padding': '10px'}),

            dash_table.DataTable(
                id='tabla_resumen',
                columns=[
                    {'name': 'Variable', 'id': 'variable'},
                    {'name': 'Valor', 'id': 'valor'}
                ],
                data=[
                    {'variable': 'Número de tiendas', 'valor': 54},
                    {'variable': 'Número de productos', 'valor': 33},
                    {'variable': 'Número de Estados', 'valor': 24},
                    {'variable': 'Número de meses', 'valor': 12}
                ],
                style_table={'width': '50%', 'margin': 'auto'},
                style_cell={
                    'textAlign': 'center',
                    'fontFamily': 'Times New Roman',
                    'color': 'green'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                },
                style_data={
                    'whiteSpace': 'normal',
                    'height': 'auto',
                    'lineHeight': '15px'
                }
            ),
            html.Div(style={'height': '20px'}),
            dcc.Graph(figure=grafica_1()),
            html.Div(style={'height': '20px'}),
            dcc.Graph(figure=grafica_2()),
            html.Div(style={'height': '20px'}),
            dcc.Graph(figure=grafica_3()),
            html.Div(style={'height': '20px'}),
            dcc.Graph(figure=grafica_4()),
            html.Div(style={'height': '20px'}),
            dcc.Graph(figure=grafica_5()),
            html.Div(style={'height': '20px'}),
            dcc.Graph(figure=grafica_6()),
            html.Div(style={'height': '20px'}),
            dcc.Graph(figure=grafica_7())
        ])

    elif tab == 'tab_2':
        return html.Div(children=[
   html.Div([
        html.H1('Visualización por tienda', style={'border': '2px solid #A1E5A4', 'padding': '10px'}),

        html.H2('Seleccionar tienda:'),
        dcc.Dropdown(id='store-dropdown',
                     options=[{'label': i, 'value': i} for i in sorted(df['store_nbr'].unique())],
                     value=1),

        html.H2('Número total de ventas por año'),
        dcc.Graph(id='total-sales'),

        html.H2('Número total de productos vendidos'),
        dcc.Graph(id='total-products'),

        html.H2('Número total de productos vendidos en promoción'),
        dcc.Graph(id='total-products-promotion'),

    ], style={'width': '100%', 'height': '80vh', 'display': 'inline-block'}),
])

    elif tab == 'tab_3':
      return html.Div(children=[
          html.Div([
              html.H1('Visualización por estado', style={'border': '2px solid #A1E5A4', 'padding': '10px'}),
              html.H2('Seleccionar estado:'),
              dcc.Dropdown(
                  id='state-dropdown',
                  options=[{'label': i, 'value': i} for i in df['state'].unique()],
                  value='Pichincha'
              ),
              dcc.Graph(id='total-transactions'),
          ], style={'width': '100%', 'height': '80vh', 'display': 'inline-block', 'margin-right': '10px'}),

          html.Div([
              html.H2(children='Ranking de tiendas con más ventas'),
              dcc.Graph(id='ranking-store-sales'),
          ], style={'width': '100%', 'height': '80vh', 'display': 'inline-block', 'margin-right': '10px'}),

          html.Div([
              html.H2(children='Producto más vendido en cada tienda'),
              html.H3('En todos los Estados el producto más vendido en cada tienda es de tipo "Grocery".'),
              dcc.Graph(id='most-selled-product'),
          ], style={'width': '100%', 'height': '80vh', 'display': 'inline-block'}),
      ])
    elif tab == 'tab_4':
         return [html.Div([
            html.H1('Visualización de aspectos generales', style={'border': '2px solid #A1E5A4', 'padding': '10px'}),
            html.H2('Evolución temporal de la empresa'),
            dcc.Graph(figure=grafica_8()),
            html.H2('El estado con más ventas es Pichincha'),
            dcc.Graph(figure=grafica_9()),
            html.Div(style={'height': '20px'}),
            dcc.Graph(figure=grafica_10())

        ])]

Finalmente ejecutamos el servidor de la aplicación y Dash lanza el servidor local donde podemos interactuar con las visualizaciones.

In [10]:
app.run_server(debug=True, jupyter_mode="external")

Dash app running on:


<IPython.core.display.Javascript object>